# Models Test

in /courses/tests.py

<pre>
 1 from django.test import TestCase
  2 from django.utils import timezone
  3
  4 from .models import Course
  5
  6
  7 class CourseModelTests(TestCase):
  8     def test_course_creationg(self):
  9         course = Course.objects.create(
 10             title = "Python regular Expression",
 11             description = "Learn to write regular expressions in Python"
 12         )
 13         now = timezone.now()
 14         self.assertLess(course.created_at, now)
 </pre>



- python manage.py test


Creating test database for alias 'default'...
System check identified no issues (0 silenced).
.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
Destroying test database for alias 'default'...
dockeruser@4fc6563981f2:~/hostname/workspace/git/kaden/dss/Practice/treehouse/learning_site$

from django.test import TestCase

from .models import Writer

class WriterModelTestCase(TestCase):
    '''Tests for the Writer model'''
    def test_attribute_in(self):
        writer = Writer.objects.create(
            name = "Kadencho",
            email = "dfdf@gmail.com",
            bio = "that's really awesome to be"
        )
        self.assertIn(writer.email, writer.mailto())

# View Test
you can test URL and view at the same time
URl and view are so tightly bound in django, so it is preferable to test both together unlike other testing which prefers one-test-for-one-function rule.

using setUp method start beginning of the each test, and it just creates some things for us

django.core.urlresolvers.reverse() takes a URL name and reverses it to the correct URL string. More information

self.client acts like a web browser and lets you make requests to URLs, both inside and outside of your Django project.

assertEqual(a, b) checks that a and b are equal to each other.

assertIn(a, b) checks that a is contained in b.


https://docs.djangoproject.com/en/1.8/ref/urlresolvers/#reverse

1 from django.core.urlresolvers import reverse
  2 from django.test import TestCase
  3 from django.utils import timezone
  4
  5 from .models import Course, Step
  6
  7
  8 class CourseModelTests(TestCase):
  9     def test_course_creation(self):
 10         course = Course.objects.create(
 11             title = "Python regular Expression",
 12             description = "Learn to write regular expressions in Python"
 13         )
 14         now = timezone.now()
 15         self.assertLess(course.created_at, now)
 16
 17
 18 class StepModelTests(TestCase):
 19     def setUp(self):
 20         self.course = Course.objects.create(
 21             title = "Python Testing",
 22             description = "Learn to write tests in Python"
 23         )
 24
 25     def test_step_creation(self):
 26         step = Step.objects.create(
 27             title = "Introduction to Doctests",
 28             description = "Learn to write tests in your docstrings",
 29             course = self.course
 30         )
 31         self.assertIn(step, self.course.step_set.all())
 32
 33
 34 class CourseViewsTests(TestCase):
 35     def setUp(self):
 36         self.course = Course.objects.create(
 37             title = "Python Testing",
 38             description = "Learn to write tests in Python",
 39         )
 40         self.course2 = Course.objects.create(
 41             title = "New Coruse",
 42             description = "A new course"
 43         )
 44         self.step = Step.objects.create(
 45             title = "Introduction to Doctests",
 46             description = "Learn to write test in your docstrings.",
 47             course = self.course
 48         )
 49
 50     def test_course_list_view(self):
 51         resp = self.client.get(reverse('courses:list'))
 52         self.assertEqual(resp.status_code, 200)
 53         self.assertIn(self.course, resp.context['courses'])
 54         self.assertIn(self.course2, resp.context['courses'])

additional 2 tests for the other views. Points are on the reverse's arguments using args & kwargs

 56     def test_course_detail_view(self):
 57         resp = self.client.get(reverse('courses:detail', args=[self.course.pk])    `)
 58         self.assertEqual(resp.status_code, 200)
 59         self.assertEqual(self.course, resp.context['course'])
 60
 61     def test_step_detail_view(self):
 62         resp = self.client.get(reverse('courses:step', kwargs={'course_pk' : se    lf.course.pk, 'step_pk' : self.step.pk}))
 63         self.assertEqual(resp.status_code, 200)
 64         self.assertEqual(self.step, resp.context['step'])`

# Template Test
- assertTemplateUsed
- assertContains

in /courses/tests.py, add 2 lines at the end of test_course_list_view

we're gonna test the 'structure' or 'string' not html which could be changed by other teams


<pre>

 50     def test_course_list_view(self):
 51         resp = self.client.get(reverse('courses:list'))
 52         self.assertEqual(resp.status_code, 200)
 53         self.assertIn(self.course, resp.context['courses'])
 54         self.assertIn(self.course2, resp.context['courses'])
 55         self.assertTemplateUsed(resp, 'courses/course_list.html')
 56         self.assertContains(resp, self.course.title)
 
 </pre>

## Treehouse challenge quiz example

import datetime

from django.core.urlresolvers import reverse
from django.test import TestCase

from .models import Article, Writer

class ArticleDetailViewTestCase(TestCase):
    '''Tests for the Article detail view'''

    def setUp(self):
        self.writer = Writer.objects.create(
            name='Kenneth Love',
            email='kenneth@teamtreehouse.com',
            bio='Your friendly, local Python teacher'
        )
        self.article = Article.objects.create(
          writer=self.writer,
          headline='Article 0',
          content='Something about 0',
          publish_date=datetime.datetime.today()
        )

    def test_detail_template(self):
        '''Make sure the `articles/article_detail.html` template is used'''
        resp = self.client.get(reverse('articles:detail',kwargs={'pk':self.article.pk}))
        self.assertEqual(resp.status_code, 200)
        self.assertEqual(self.article, resp.context['article'])
        self.assertTemplateUsed(resp, 'articles/article_detail.html')


    def test_detail_template_writer(self):
        '''Make sure the article writer's name is in the rendered output'''
        resp = self.client.get(reverse('articles:detail', kwargs={'pk':self.article.pk}))  # <-- changed to article:detail
        self.assertEqual(resp.status_code, 200)
        self.assertEqual(self.article, resp.context['article'])  # <-- changed to self.article
        self.assertContains(resp, self.article.writer.name)  # <-- add 'assertConains' check